In [2]:
import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
query = '''
SELECT
    MAX(Equipamento.Descricao) AS descricaoEquipamento
    , MAX(InstalacaoEletrica.Descricao) AS descricaoInstalacaoEletrica
    , MAX(gc.Descricao) AS GrupoDeCalculo
    , MAX(Calculo.Codigo) AS CodigoCalculo
    , MAX(Calculo.Descricao) AS DescricaoCalculo
    , ExecucaoCalculoResultado.Resultado 
	, CAST(ExecucaoCalculoResultado.UltimaAtualizacao AS DATE) AS UltimaAtualizacaoCalculo
FROM EngineCalculo.CalculoResultado As ExecucaoCalculoResultado
INNER JOIN EngineCalculo.CalculoResultadoEquipamento 
	ON CalculoResultadoEquipamento.CalculoId = ExecucaoCalculoResultado.CalculoId 
    AND CalculoResultadoEquipamento.JobId = ExecucaoCalculoResultado.JobId
INNER JOIN EngineCalculo.Calculo
	ON Calculo.Id = ExecucaoCalculoResultado.CalculoId
INNER JOIN EngineCalculo.CalculoResultadoVariavel As EntradaVariavel 
	ON ExecucaoCalculoResultado.JobId = EntradaVariavel.JobId
INNER JOIN EngineCalculo.RevisaoVariavel As RevisaoVariavel 
    ON RevisaoVariavel.VariavelId = EntradaVariavel.VariavelId 
	AND RevisaoVariavel.Revisao = EntradaVariavel.Revisao
INNER JOIN [EngineCalculo].[Variavel] 
	ON Variavel.Id = EntradaVariavel.VariavelId 
INNER JOIN Equipamento
	ON Equipamento.Id = CalculoResultadoEquipamento.EquipamentoId
INNER JOIN EquipamentoInstalacaoEletrica 
	ON Equipamento.Id = EquipamentoInstalacaoEletrica.EquipamentoId
INNER JOIN InstalacaoEletrica
	ON  InstalacaoEletrica.Id = EquipamentoInstalacaoEletrica.InstalacaoEletricaId
INNER JOIN Familia AS f 
	ON f.Id = Equipamento.FamiliaId
INNER JOIN EngineCalculo.CategoriaVariavel AS cv 
	ON cv.Id = Variavel.CategoriaVariavelId
LEFT JOIN EngineCalculo.GrupoCalculo AS gc 
	ON gc.Id = Calculo.GrupoCalculoId
WHERE 
	Calculo.Codigo IN ('IE_TR' ,'I2_TR')
    AND ExecucaoCalculoResultado.UltimaAtualizacao >= '2023-06-07'
    AND ExecucaoCalculoResultado.UltimaAtualizacao <= '2023-07-31'
	AND Equipamento.Id = 132
GROUP BY 
	ExecucaoCalculoResultado.Resultado 
	, CAST(ExecucaoCalculoResultado.UltimaAtualizacao AS DATE)
ORDER BY 
	CAST(ExecucaoCalculoResultado.UltimaAtualizacao AS DATE)
    , MAX(InstalacaoEletrica.Descricao)
	, MAX(Equipamento.Descricao) '''
resultado = cursor.execute(query)
registro = resultado.fetchall()


In [4]:
colunas = [column[0] for column in cursor.description]

In [5]:
colunas

['descricaoEquipamento',
 'descricaoInstalacaoEletrica',
 'GrupoDeCalculo',
 'CodigoCalculo',
 'DescricaoCalculo',
 'Resultado',
 'UltimaAtualizacaoCalculo']